In [9]:
%reload_ext autoreload
%autoreload 2
!pip install -r requirements.txt
!playwright install

In [1]:
from server.llm import (parse_markdown_output, create_prompt_from_template, LlmClient)
from server.scraper import WebScraper
import nest_asyncio
import asyncio
import time
import os

scraper = WebScraper()
llm = LlmClient()

async def generate_variations(original_url, preview_url, project_dir, selector):

    os.makedirs(f"{project_dir}/generated", exist_ok=True)

    print('Getting the HTML and screenshot of the original page...')
    original_html, original_screenshot = await scraper.get_html_and_screenshot(original_url, selector)
    
    for iteration in range(0, 6):

        print(f'Running iteration {iteration}...')

        print('Running the assessment...')
        goals = [
            "Improve conversion rate",
            "Make the styles more consistent with the brand",
            "Improve the overall look and feel of the form"
        ]
        assessment_prompt = create_prompt_from_template(
            "server/generation_strategies/generate-css/prompts/assessment.txt", 
            goals=goals,
            html=original_html.strip()
        )
        instructions = llm.get_completions(assessment_prompt, [original_screenshot])
        print(instructions)

        print('Generating CSS variation...')
        generation_prompt = create_prompt_from_template(
            "server/generation_strategies/generate-css/prompts/generation.txt",
            instructions=instructions,
            html=original_html.strip()
        )
        raw_output = llm.get_completions(generation_prompt, [original_screenshot])
        parsed_output = parse_markdown_output(raw_output)
        with open(project_dir + f'generatedStyles.css', 'w') as text_file:
            text_file.write('\n'.join(parsed_output['css']))

        print("Sleeping for 3 second...")
        time.sleep(3)

        print('Getting the HTML and screenshot of the preview page...')
        generated_html, generated_screenshot = await scraper.get_html_and_screenshot(preview_url + '?generatedStyles', selector)

        print('Assessing the quality of the generated styles...')
        verification_prompt = create_prompt_from_template(
            "server/generation_strategies/generate-css/prompts/verification.txt",
            html=generated_html.strip()
        )
        adjustments = llm.get_completions(verification_prompt, [generated_screenshot]) 
        print(adjustments)

        print('Applying adjustments...')
        adjustment_prompt = create_prompt_from_template(
            "server/generation_strategies/generate-css/prompts/adjustment.txt",
            adjustments=adjustments,
            html=generated_html.strip()
        )
        raw_output = llm.get_completions(adjustment_prompt, [generated_screenshot])
        parsed_output = parse_markdown_output(raw_output)

        with open(project_dir + f'generatedStyles.css', 'w') as text_file:
            text_file.write('\n'.join(parsed_output['css']))

        with open(project_dir + f'generated/styles{iteration}.css', 'w') as text_file:
            text_file.write('\n'.join(parsed_output['css']))

original_url = "https://main--wknd--hlxsites.hlx.live/"
preview_url = "http://localhost:3000/"
selector = '.featured-article.block'
project_dir = '/Users/tsaplin/Work/Sources/mystique-wknd/'

nest_asyncio.apply()
asyncio.get_event_loop().run_until_complete(generate_variations(original_url, preview_url, project_dir, selector))


Getting the HTML and screenshot of the original page...
Running iteration 0...
Running the assessment...


KeyboardInterrupt: 